In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [23]:
import numpy as np

train_data,test_data =imdb['train'],imdb['test']

training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [13]:
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok="<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [15]:
# Reverse word index 

reverse_word_index=dict((value,key) for (key,value) in word_index.items())

def decode_review(text):
    return " ".join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[0]))
print(training_sentences[0])

? ? b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pa

In [30]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [31]:
num_epochs=10
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded,testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 105us/sample - loss: 0.4895 - acc: 0.7434 - val_loss: 0.3469 - val_acc: 0.8474
Epoch 2/10
25000/25000 [==============================] - 2s 93us/sample - loss: 0.2437 - acc: 0.9052 - val_loss: 0.3690 - val_acc: 0.8384
Epoch 3/10
25000/25000 [==============================] - 2s 87us/sample - loss: 0.1002 - acc: 0.9734 - val_loss: 0.4523 - val_acc: 0.8231
Epoch 4/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.0262 - acc: 0.9963 - val_loss: 0.5259 - val_acc: 0.8244
Epoch 5/10
25000/25000 [==============================] - 3s 100us/sample - loss: 0.0057 - acc: 0.9996 - val_loss: 0.5908 - val_acc: 0.8273
Epoch 6/10
25000/25000 [==============================] - 3s 103us/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.6407 - val_acc: 0.8276
Epoch 7/10
25000/25000 [==============================] - 2s 94us/sample - loss: 9.0188e-04 - acc: 1.0000 - val_l

In [32]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)           # shape: (vocab_size,embedding_dim)

(10000, 16)


In [33]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

**try yourself** [Embedding Projector](https://projector.tensorflow.org/)